# ORIGINAL MAPPING & CREATION OF SINGLE TILES

In [ ]:
import math

# Quadrant remapping: 1→2, 2→4, 3→1, 4→3
QUADRANT_MAP = {1: 2, 2: 4, 3: 1, 4: 3}

# Geographic bounds (degrees)
minLat, maxLat = 34.07201, 34.21606
minLon, maxLon = 77.45396, 77.62802

# Split into 16x16 grid (256 tiles)
num_rows = 8
num_cols = 8

# Height bounds (from your terrain_bbox)
minHeight = 3162.0
maxHeight = 4685.0
tileset = {
    "asset": {
        "version": "1.0",
        "gltfUpAxis": "Y"
    },
    "geometricError": 1000,
    "root": {
        "transform": [
            -0.976450790243208,
            0.21574024713393025,
            0,
            0,
            -0.12108991487890734,
            -0.5480588098176702,
            0.8276284030262898,
            0,
            0.17855275620395158,
            0.808138408162744,
            0.5612764261076174,
            0,
            1140597.9510803188,
            5162401.478064311,
            3561440.7474683644,
            1
        ],
        "boundingVolume": {
            "region": [
                math.radians(minLon),
                math.radians(minLat),
                math.radians(maxLon),
                math.radians(maxLat),
                minHeight,
                maxHeight
            ]
        },
        "geometricError": 500,
        "refine": "ADD",
        "children": []
    }
}


# def compute_child_regions(minLat, maxLat, minLon, maxLon, num_rows, num_cols):
#     lat_step = (maxLat - minLat) / num_rows
#     lon_step = (maxLon - minLon) / num_cols

#     regions = []
#     for row in range(num_rows):
#         for col in range(num_cols):
#             # Calculate tile bounds in degrees
#             tile_minLat = minLat + row * lat_step
#             tile_maxLat = minLat + (row + 1) * lat_step
#             tile_minLon = minLon + col * lon_step
#             tile_maxLon = minLon + (col + 1) * lon_step

#             # Convert to radians for Cesium's region
#             west = math.radians(tile_minLon)
#             south = math.radians(tile_minLat)
#             east = math.radians(tile_maxLon)
#             north = math.radians(tile_maxLat)
#             regions.append([west, south, east, north, minHeight, maxHeight])
#     return regions

def calculate_udim(row, col):
    # Original quadrant detection
    if row < 8 and col < 8:
        quadrant = 1
    elif row < 8 and col >= 8:
        quadrant = 2
    elif row >= 8 and col < 8:
        quadrant = 3
    else:
        quadrant = 4

    # Get remapped quadrant
    new_quadrant = QUADRANT_MAP[quadrant]
    
    # Calculate local coordinates within original quadrant
    local_i = row % 8
    local_j = col % 8
    
    # Transform coordinates for 90° CCW rotation
    if new_quadrant == 2:  # Former quadrant 1
        udim_i = 7 - local_j  # Flip Y axis
        udim_j = local_i
    elif new_quadrant == 4:  # Former quadrant 2
        udim_i = 7 - local_j
        udim_j = local_i
    elif new_quadrant == 1:  # Former quadrant 3
        udim_i = 7 - local_j
        udim_j = local_i
    else:  # new_quadrant == 3 (former 4)
        udim_i = 7 - local_j
        udim_j = local_i

    return f"{new_quadrant}_{1001 + udim_i * 10 + udim_j}"

def compute_child_regions(minLat, maxLat, minLon, maxLon, num_rows, num_cols):
    lat_step = (maxLat - minLat) / num_rows
    lon_step = (maxLon - minLon) / num_cols

    regions = []
    for row in range(num_rows):
        for col in range(num_cols):
            # Calculate tile bounds in degrees
            tile_minLat = minLat + row * lat_step
            tile_maxLat = tile_minLat + lat_step
            tile_minLon = minLon + col * lon_step
            tile_maxLon = tile_minLon + lon_step

            # Convert to radians for Cesium's region
            west = math.radians(tile_minLon)
            south = math.radians(tile_minLat)
            east = math.radians(tile_maxLon)
            north = math.radians(tile_maxLat)

            regions.append([west, south, east, north, minHeight, maxHeight])
    return regions

child_regions = compute_child_regions(minLat, maxLat, minLon, maxLon, num_rows, num_cols)
# Add child tiles (replace "tile_{row}_{col}.glb" with your GLB filenames)
# Generate tileset with corrected mapping
for i, region in enumerate(child_regions):
    row = i // num_cols
    col = i % num_cols
    
    # Get properly rotated UDIM
    udim = calculate_udim(row, col)
    
    tileset["root"]["children"].append({
        "boundingVolume": {"region": region},
        "geometricError": 200,
        "content": {"uri": f"{udim}_.glb"},
        "metadata": {"original_row": row, "original_col": col}
    })
# Save to JSON
import json
with open("tileset-8x8.json", "w") as f:
    json.dump(tileset, f, indent=4)
    

# CHILDREN MODIFICATION - LOD REGION CALCULATION

In [4]:
import math
import json

# Quadrant remapping: 1→2, 2→4, 3→1, 4→3
QUADRANT_MAP = {1: 2, 2: 4, 3: 1, 4: 3}

# Geographic bounds (degrees)
minLat, maxLat = 34.07201, 34.21606
minLon, maxLon = 77.45396, 77.62802

# Height bounds
minHeight = 3162.0
maxHeight = 4685.0

def calculate_udim(row, col):
    # Original quadrant detection
    if row < 8 and col < 8:
        quadrant = 1
    elif row < 8 and col >= 8:
        quadrant = 2
    elif row >= 8 and col < 8:
        quadrant = 3
    else:
        quadrant = 4

    # Get remapped quadrant
    new_quadrant = QUADRANT_MAP[quadrant]
    
    # Calculate local coordinates within original quadrant
    local_i = row % 8
    local_j = col % 8
    
    # Transform coordinates for 90° CCW rotation
    if new_quadrant == 2:  # Former quadrant 1
        udim_i = 7 - local_j  # Flip Y axis
        udim_j = local_i
    elif new_quadrant == 4:  # Former quadrant 2
        udim_i = 7 - local_j
        udim_j = local_i
    elif new_quadrant == 1:  # Former quadrant 3
        udim_i = 7 - local_j
        udim_j = local_i
    else:  # new_quadrant == 3 (former 4)
        udim_i = 7 - local_j
        udim_j = local_i

    return f"{new_quadrant}_{1001 + udim_i * 10 + udim_j}"


def compute_child_regions(minLat, maxLat, minLon, maxLon, num_rows, num_cols):
    lat_step = (maxLat - minLat) / num_rows
    lon_step = (maxLon - minLon) / num_cols

    regions = []
    for row in range(num_rows):
        for col in range(num_cols):
            # Calculate tile bounds in degrees
            tile_minLat = minLat + row * lat_step
            tile_maxLat = tile_minLat + lat_step
            tile_minLon = minLon + col * lon_step
            tile_maxLon = tile_minLon + lon_step

            # Convert to radians for Cesium's region
            west = math.radians(tile_minLon)
            south = math.radians(tile_minLat)
            east = math.radians(tile_maxLon)
            north = math.radians(tile_maxLat)

            regions.append([west, south, east, north, minHeight, maxHeight])
    return regions


# Initialize tileset with root
tileset = {
    "asset": {"version": "1.0", "gltfUpAxis": "Y"},
    "geometricError": 1000,
    "root": {
        "transform": [
            -0.976450790243208, 0.21574024713393025, 0, 0,
            -0.12108991487890734, -0.5480588098176702, 0.8276284030262898, 0,
            0.17855275620395158, 0.808138408162744, 0.5612764261076174, 0,
            1140597.9510803188, 5162401.478064311, 3561440.7474683644, 1
        ],
        "boundingVolume": {
            "region": [
                math.radians(minLon), math.radians(minLat),
                math.radians(maxLon), math.radians(maxLat),
                minHeight, maxHeight
            ]
        },
        "geometricError": 800,
        "refine": "ADD",
        "children": []
    }
}

# Generate HLOD levels
def build_hierarchy():
    # Level 4: Leaf nodes (16x16 = 256 tiles)
    level4_regions = compute_child_regions(minLat, maxLat, minLon, maxLon, 16, 16)
    level4_tiles = []
    for i, region in enumerate(level4_regions):
        row = i // 16
        col = i % 16
        udim = calculate_udim(row, col)
        level4_tiles.append({
            "boundingVolume": {"region": region},
            "geometricError": 50,
            "content": {"uri": f"{udim}_.glb"},
            "metadata": {"original_row": row, "original_col": col}
        })

    # Level 3: 8x8 tiles (parents of 4x4 level4 tiles)
    level3_regions = compute_child_regions(minLat, maxLat, minLon, maxLon, 8, 8)
    level3_tiles = []
    for i, region in enumerate(level3_regions):
        row3 = i // 8
        col3 = i % 8
        children = []
        for dr in [0, 1]:
            for dc in [0, 1]:
                child_idx = (row3 * 2 + dr) * 16 + (col3 * 2 + dc)
                children.append(level4_tiles[child_idx])
        level3_tiles.append({
            "boundingVolume": {"region": region},
            "geometricError": 100,
            "children": children
        })

    # Level 2: 4x4 tiles (parents of level3 tiles)
    level2_regions = compute_child_regions(minLat, maxLat, minLon, maxLon, 4, 4)
    level2_tiles = []
    for i, region in enumerate(level2_regions):
        row2 = i // 4
        col2 = i % 4
        children = []
        for dr in [0, 1]:
            for dc in [0, 1]:
                child_idx = (row2 * 2 + dr) * 8 + (col2 * 2 + dc)
                children.append(level3_tiles[child_idx])
        level2_tiles.append({
            "boundingVolume": {"region": region},
            "geometricError": 200,
            "children": children
        })

    # Level 1: 2x2 tiles (parents of level2 tiles)
    level1_regions = compute_child_regions(minLat, maxLat, minLon, maxLon, 2, 2)
    level1_tiles = []
    for i, region in enumerate(level1_regions):
        row1 = i // 2
        col1 = i % 2
        children = []
        for dr in [0, 1]:
            for dc in [0, 1]:
                child_idx = (row1 * 2 + dr) * 4 + (col1 * 2 + dc)
                children.append(level2_tiles[child_idx])
        level1_tiles.append({
            "boundingVolume": {"region": region},
            "geometricError": 400,
            "children": children
        })

    # Attach level1 tiles to root
    tileset["root"]["children"] = level1_tiles

build_hierarchy()

# Save to JSON
with open("tileset-all.json", "w") as f:
    json.dump(tileset, f, indent=4)

# RENAMING OF LODs

In [ ]:
import math
import json

# Quadrant remapping: 1→2, 2→4, 3→1, 4→3
QUADRANT_MAP = {1: 2, 2: 4, 3: 1, 4: 3}

# Geographic bounds (degrees)
minLat, maxLat = 34.07201, 34.21606
minLon, maxLon = 77.45396, 77.62802

# Height bounds
minHeight = 3162.0
maxHeight = 4685.0

def calculate_udim_for_level(row, col, grid_size):
    half = grid_size // 2
    if row < half and col < half:
        quadrant = 4
    elif row < half and col >= half:
        quadrant = 2
    elif row >= half and col < half:
        quadrant = 3
    else:
        quadrant = 1

    new_quadrant = QUADRANT_MAP[quadrant]
    
    local_i = row % half
    local_j = col % half
    
    # Transform coordinates for 90° CCW rotation
    udim_i = (half - 1) - local_j
    udim_j = local_i

    return f"{new_quadrant}_{1001 + udim_i * 10 + udim_j}"

def compute_child_regions(minLat, maxLat, minLon, maxLon, num_rows, num_cols):
    # Existing implementation remains the same
    lat_step = (maxLat - minLat) / num_rows
    lon_step = (maxLon - minLon) / num_cols

    regions = []
    for row in range(num_rows):
        for col in range(num_cols):
            tile_minLat = minLat + row * lat_step
            tile_maxLat = tile_minLat + lat_step
            tile_minLon = minLon + col * lon_step
            tile_maxLon = tile_minLon + lon_step

            west = math.radians(tile_minLon)
            south = math.radians(tile_minLat)
            east = math.radians(tile_maxLon)
            north = math.radians(tile_maxLat)

            regions.append([west, south, east, north, minHeight, maxHeight])
    return regions

tileset = {
    "asset": {"version": "1.0", "gltfUpAxis": "Y"},
    "geometricError": 1000,
    "root": {
        "transform": [
            -0.976450790243208, 0.21574024713393025, 0, 0,
            -0.12108991487890734, -0.5480588098176702, 0.8276284030262898, 0,
            0.17855275620395158, 0.808138408162744, 0.5612764261076174, 0,
            1140597.9510803188, 5162401.478064311, 3561440.7474683644, 1
        ],
        "boundingVolume": {
            "region": [
                math.radians(minLon), math.radians(minLat),
                math.radians(maxLon), math.radians(maxLat),
                minHeight, maxHeight
            ]
        },
        "geometricError": 800,
        "refine": "ADD",
        "children": []
    }
}

def build_hierarchy():
    # Level 4: Leaf nodes (16x16)
    level4_regions = compute_child_regions(minLat, maxLat, minLon, maxLon, 16, 16)
    level4_tiles = []
    for i, region in enumerate(level4_regions):
        row = i // 16
        col = i % 16
        udim = calculate_udim_for_level(row, col, 16)
        level4_tiles.append({
            "boundingVolume": {"region": region},
            "geometricError": 50,
            "content": {"uri": f"{udim}.glb"},
            "metadata": {"original_row": row, "original_col": col}
        })

    # Level 3: 8x8 tiles
    level3_regions = compute_child_regions(minLat, maxLat, minLon, maxLon, 8, 8)
    level3_tiles = []
    for i, region in enumerate(level3_regions):
        row3 = i // 8
        col3 = i % 8
        children = []
        for dr in [0, 1]:
            for dc in [0, 1]:
                child_idx = (row3 * 2 + dr) * 16 + (col3 * 2 + dc)
                children.append(level4_tiles[child_idx])
        udim = calculate_udim_for_level(row3, col3, 8)
        level3_tiles.append({
            "boundingVolume": {"region": region},
            "geometricError": 100,
            "content": {"uri": f"{udim}_LOD2.glb"},
            "children": children
        })

    # Level 2: 4x4 tiles
    level2_regions = compute_child_regions(minLat, maxLat, minLon, maxLon, 4, 4)
    level2_tiles = []
    for i, region in enumerate(level2_regions):
        row2 = i // 4
        col2 = i % 4
        children = []
        for dr in [0, 1]:
            for dc in [0, 1]:
                child_idx = (row2 * 2 + dr) * 8 + (col2 * 2 + dc)
                children.append(level3_tiles[child_idx])
        udim = calculate_udim_for_level(row2, col2, 4)
        level2_tiles.append({
            "boundingVolume": {"region": region},
            "geometricError": 200,
            "content": {"uri": f"{udim}_LOD3.glb"},
            "children": children
        })

    # Level 1: 2x2 tiles
    level1_regions = compute_child_regions(minLat, maxLat, minLon, maxLon, 2, 2)
    level1_tiles = []
    for i, region in enumerate(level1_regions):
        row1 = i // 2
        col1 = i % 2
        children = []
        for dr in [0, 1]:
            for dc in [0, 1]:
                child_idx = (row1 * 2 + dr) * 4 + (col1 * 2 + dc)
                children.append(level2_tiles[child_idx])
        udim = calculate_udim_for_level(row1, col1, 2)
        level1_tiles.append({
            "boundingVolume": {"region": region},
            "geometricError": 400,
            "content": {"uri": f"{udim}_LOD4.glb"},
            "children": children
        })

    # Attach level1 tiles to root
    tileset["root"]["children"] = level1_tiles

build_hierarchy()

with open("tileset-all-lod.json", "w") as f:
    json.dump(tileset, f, indent=4)